In [55]:
from gs_quant.target.instrument import IRSwaption
from gs_quant.risk import IRAnnualImpliedVol, Price, IRFwdRate, IRAnnualATMImpliedVol, IRDelta
from gs_quant.session import GsSession, Environment
from gs_quant.common import MeasureScenario
from gs_quant.markets import PricingContext
import datetime as dt

In [56]:
# external users should substitute their client id and secret; please skip this step if using internal jupyterhub
GsSession.use(Environment.QA, client_id=None, client_secret=None, scopes=('run_analytics',))

MeasureScenario class allows us to override inputs such as spot or volatility when pricing an instrument. Here are some examples of how to use it with Interest rate swaption.

In [ ]:
# Initial IRSwaption
swaption = IRSwaption(notional_currency='EUR', effective_date='1y', termination_date='10y')
swaption.to_dict()

In [ ]:
# Override a single input with MeasureScenario context manager
with MeasureScenario(IRFwdRate, 0.024):
    risk = swaption.calc((IRAnnualImpliedVol, Price, IRFwdRate, IRAnnualATMImpliedVol))
risk.to_frame()

We can see that the Interest rate forward rate value was overridden, and other variables are re-calculated to account for the new rate.

We might also be interested in comparing the forward rate between the original, up, and down override scenarios.

In [ ]:
init = swaption.calc((IRDelta(aggregation_level='Type'), Price, IRFwdRate))
init.to_frame()

In [60]:
with MeasureScenario(IRFwdRate, init[IRFwdRate] + 0.0001):
    up = swaption.calc((Price, IRFwdRate))

with MeasureScenario(IRFwdRate, init[IRFwdRate] - 0.0001):
    down = swaption.calc((Price, IRFwdRate))

In [ ]:
print(init[IRFwdRate])
print(up[IRFwdRate])
print(down[IRFwdRate])

The forward rates are close in 3 scenarios, which is expected as the swaption expiry date is in 1 year only. 

We can also override multiple inputs.


In [ ]:
# Override multiple inputs (with nested context managers)
with MeasureScenario(IRFwdRate, 0.028):
    with MeasureScenario(IRAnnualImpliedVol, 0.009):
        risk = swaption.calc((IRAnnualImpliedVol, Price, IRFwdRate, IRAnnualATMImpliedVol))
risk.to_frame()

We can also combine the measure override with other context managers, such as PricingContext


In [ ]:
# Combine with PricingContext context manager
with PricingContext(dt.date(2025, 7, 5)):
    with MeasureScenario(IRAnnualImpliedVol, 0.002):
        risk = swaption.calc((IRAnnualImpliedVol, Price, IRFwdRate, IRAnnualATMImpliedVol))
risk.result().to_frame()